In [1]:
import requests
import pandas as pd
import arrow
import datetime
import numpy as np

import warnings

import mplfinance as mpf
import matplotlib.pyplot as plt

In [2]:
company =[ "CIPLA","TCS","INFY","INFRATEL","ITC","HCLTECH","VEDL","WIPRO","BRITANNIA","UPL","RELIANCE","TECHM","M_M","SUNPHARMA","NESTLEIND",
"HINDUNILVR","DRREDDY","TATASTEEL","BHARTIARTL","NTPC","ONGC","TITAN","POWERGRID","HINDALCO","ASIANPAINT","TATAMOTORS","SHREECEM",
"GRASIM","IOC","HEROMOTOCO","BAJAJFINSV","ADANIPORTS","EICHERMOT","COALINDIA","JSWSTEEL","KOTAKBANK","LT","GAIL","HDFCBANK",
"SBIN","BAJAJ-AUTO","BAJFINANCE","BPCL","HDFC","MARUTI","ULTRACEMCO","AXISBANK","ICICIBANK","ZEEL","INDUSINDBK" ]

dates = ['2020-05-11','2020-05-12','2020-05-13','2020-05-14','2020-05-15','2020-05-18']
# 23-04 & 30-04 Missing data between 10.25 - 11.25

In [3]:
def style():
    mc = mpf.make_marketcolors(
                            up='tab:green',down='tab:red',
                            edge='black',
                            wick={'up':'green','down':'red'},
                            volume={'up':'green','down':'red'},
                           )

    return mpf.make_mpf_style(marketcolors=mc)

def bollvwap(ndata):
    return [mpf.make_addplot(ndata.MBB,color='black'),mpf.make_addplot(ndata.UBB,color='black'),mpf.make_addplot(ndata.LBB,color='black'),mpf.make_addplot(ndata.VWAP, color='blue')]

def boll(ndata):
    return [mpf.make_addplot(ndata.MBB,color='black'),mpf.make_addplot(ndata.UBB,color='black'),mpf.make_addplot(ndata.LBB,color='black')]

def plotatr(ndata):
    fig = plt.figure()
    fig.set_figheight(10)
    fig.set_figwidth(10)
    plt.plot(ndata.ATR)
    plt.ylabel('ATR')
    
def plotrsi(ndata):
    fig = plt.figure()
    fig.set_figheight(10)
    fig.set_figwidth(10)
    plt.plot(ndata.RSI)
    plt.ylabel('RSI')
    
def dayplot(rdata):
    ndata  = rdata.copy()
    ndata.rename(columns={'Datetime':'Date'}, inplace=True)
    ndata.set_index('Date', inplace=True)
    ndata.index = pd.to_datetime(ndata.index)
    mpf.plot(ndata,addplot=boll(ndata),type='candle',volume=True,style=style(),figratio=(20,20))
    #plotrsi(ndata)
    #plotatr(ndata)

In [4]:
def daydata(ndata, ndate):
    i = 0
    while(True):
        if(ndata['Datetime'][ndata.index == i].values[0][0:10] == ndate):
            break
        else:
            i += 1
    st = i
    
    if(ndate == ndata.tail(1).Datetime.values[0][0:10]):
        en = len(ndata)
    else:
        while(True):
            if(ndata['Datetime'][ndata.index == i].values[0][0:10] != ndate):
                break
            else:
                i += 1
                en = i
        
    return ndata[st:en]

In [5]:
def simulate(data, date, name, good, bad):
    scanstart = '09:18:00'
    scanend = '10:00:00'
    day = daydata(data, date)
    #print(date+' '+scanstart)
    
    if(day[day['Datetime']==date+' '+scanstart].empty or day[day['Datetime']==date+' '+scanend].empty):
        print(date+": Data missing")
        return
    
    startind = day[day['Datetime']==date+' '+scanstart].index.tolist()[0]
    endind = day[day['Datetime']==date+' '+scanend].index.tolist()[0]
    ind = startind
    a=0

    while(True):
        
        if(ind == None):
            print("Start time not found in day's data")
            break
        
        temp = day[day.index == ind]
        templag1 = day[day.index == (ind-1)]
        templag2 = day[day.index == (ind-2)]
        templead1 = day[day.index == (ind+1)]
        templead2 = day[day.index == (ind+2)]
        templead3 = day[day.index == (ind+3)]
        templead4 = day[day.index == (ind+4)]
    
        if((temp.index == endind)[0]):
            break
        
        #condition1 = ((temp.Close-temp.Open)>0).tolist()[0] and ((templag1.Close-templag1.Open)>0).tolist()[0] and ((templag2.Close-templag2.Open)>0).tolist()[0]
        #condition2 = ((temp.Close.values[0] + templag1.Close.values[0] + templag2.Close.values[0]) - (temp.Open.values[0] + templag1.Open.values[0] + templag2.Open.values[0])) < 0.8*(temp.ATR.values[0] + templag1.ATR.values[0] + templag2.ATR.values[0])
        #condition3 = ((temp.Close.values[0] + templag1.Close.values[0] + templag2.Close.values[0]) - (temp.Open.values[0] + templag1.Open.values[0] + templag2.Open.values[0])) > 0.4*(temp.ATR.values[0] + templag1.ATR.values[0] + templag2.ATR.values[0])
        
        condition1 = ((temp.Close-temp.Open)>0).tolist()[0] and ((templag1.Close-templag1.Open)>0).tolist()[0] #and ((templag2.Close-templag2.Open)>0).tolist()[0]
        condition2 = ((temp.Close.values[0] + templag1.Close.values[0]) - (temp.Open.values[0] + templag1.Open.values[0])) < 2*(temp.ATR.values[0] + templag1.ATR.values[0])
        condition3 = ((temp.Close.values[0] + templag1.Close.values[0]) - (temp.Open.values[0] + templag1.Open.values[0])) > 0.6*(temp.ATR.values[0] + templag1.ATR.values[0])
         
    
        if(condition1 and condition2 and condition3 and a<1):
            #print(temp['Datetime'].values[0]+": "+name+" Long Signal")
            #print("Plotting Graph for confirmation...")
            #dayplot(day.loc[(startind-2):(startind+ 2*(endind-startind))])
            
            out1 = ((templead1.Close.values[0] + templead2.Close.values[0]) - (templead1.Open.values[0] + templead2.Open.values[0])) > .25*(templead1.ATR.values[0] + templead2.ATR.values[0])
            out2 = ((templead1.Close.values[0] + templead2.Close.values[0] + templead3.Close.values[0]) - (templead1.Open.values[0] + templead2.Open.values[0] + templead3.Open.values[0])) > .25*(templead1.ATR.values[0] + templead2.ATR.values[0] + templead3.ATR.values[0])
            out3 = ((templead1.Close.values[0] + templead2.Close.values[0] + templead3.Close.values[0] + templead4.Close.values[0]) - (templead1.Open.values[0] + templead2.Open.values[0] + templead3.Open.values[0] + templead4.Open.values[0])) > .25*(templead1.ATR.values[0] + templead2.ATR.values[0] + templead3.ATR.values[0] + templead4.ATR.values[0])
            
            if(out1 or out2 or out3):
                #print(temp['Datetime'].values[0]+": "+name+" Long Signal: Good")
                good += 1
            else:
                #print(temp['Datetime'].values[0]+": "+name+" Long Signal: Bad")
                bad += 1
            
            a += 1
            
        ind += 1
        
    return good, bad;

In [6]:
for name in company:
    good = 0
    bad = 0
    data = pd.read_csv('C:/Users/user/Stockomation/Github/Data/NIFTY 50/1MIN/'+name+'.csv')
    for event in dates:
        date = event
        good, bad = simulate(data, event, name, good, bad)
        
    print(name)
    print('Good Signals:' + str(good))
    print('Bad Signals:' + str(bad))

CIPLA
Good Signals:1
Bad Signals:5
TCS
Good Signals:0
Bad Signals:6
INFY
Good Signals:2
Bad Signals:4
INFRATEL
Good Signals:2
Bad Signals:4
ITC
Good Signals:1
Bad Signals:4
HCLTECH
Good Signals:2
Bad Signals:4
VEDL
Good Signals:2
Bad Signals:4
WIPRO
Good Signals:0
Bad Signals:5
BRITANNIA
Good Signals:1
Bad Signals:5
UPL
Good Signals:2
Bad Signals:4
RELIANCE
Good Signals:1
Bad Signals:4
TECHM
Good Signals:4
Bad Signals:1
M_M
Good Signals:1
Bad Signals:2
SUNPHARMA
Good Signals:1
Bad Signals:5
NESTLEIND
Good Signals:1
Bad Signals:4
HINDUNILVR
Good Signals:0
Bad Signals:5
DRREDDY
Good Signals:0
Bad Signals:5
TATASTEEL
Good Signals:2
Bad Signals:3
BHARTIARTL
Good Signals:2
Bad Signals:4
NTPC
Good Signals:1
Bad Signals:4
ONGC
Good Signals:1
Bad Signals:4
TITAN
Good Signals:3
Bad Signals:2
POWERGRID
Good Signals:3
Bad Signals:3
HINDALCO
Good Signals:0
Bad Signals:6
ASIANPAINT
Good Signals:1
Bad Signals:4
TATAMOTORS
Good Signals:1
Bad Signals:5
SHREECEM
Good Signals:1
Bad Signals:5
GRASIM
Good